What we need is a small parcel to work on, with a nice contourplot with critical points and minima found after initiating local method and then a 3d plot. 

In [1]:
using Pkg
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames

  Activating project at `~/Globtim.jl`


In [2]:
# Constants and Parameters
const n, a, b = 2, 4, 12
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = tref # Objective function

tref (generic function with 1 method)

One may assume that when we have access to exact evaluations, we would want to have a small $L^2$-norm tolerance `tol_l2 = 5e-4` and high probability of computing an accurate discrete $L^2$-norm `alpha= 1/10`.

We need to also return the number of samples used to generate the sample set. It is annoying that the error goes up while the degree has increased.

In [3]:
# rand_center = [2*rand()-1, 2*rand()-1]*.75; # Random center
# rand_center = [0.4694712315165298, -0.45637754560934185]
rand_center = [0.0, 0.0];

In [11]:
d = 30 # Initial Degree 
SMPL = 300 # Number of samples
TR = test_input(f, 
                dim = n,
                center=rand_center,
                GN=SMPL, 
                sample_range=scale_factor, 
                degree_max = d+4
                )
pol_cheb = Constructor(TR, d, basis=:chebyshev)
pol_lege = Constructor(TR, d, basis=:legendre);

@polyvar(x[1:n]); # Define polynomial ring 

current L2-norm: 0.0003254033403545178
current L2-norm: 0.21653386727580476


Solve the system of partial derivatives using `Msolve`. For degree `26` on tref centered at `rand_center = [0.4694712315165298, -0.45637754560934185]` and `a/b = 7//12`, `msolve runs single threaded in 5m 2.2 s. With 10 threads, it runs in 4m 48.2 s. 
The slow part of the process is the real root isolation it would seem. 

In [12]:
df_cheb = solve_and_parse(pol_cheb, x, f, TR)
sort!(df_cheb, :z, rev=true)


=== Starting MSolve Parser (dimension: 2) ===
Processed 269 points (0.009s)


Row,x1,x2,z
,Float64,Float64,Float64
1,0.0258173,-0.140992,6.28737
2,0.287448,0.170945,6.16155
3,0.286533,0.251541,6.09426
4,0.0274679,0.331984,5.83762
5,0.0274852,0.328735,5.83043
6,0.0274215,-0.288537,5.73714
7,0.286093,0.327728,5.43213
8,0.286086,0.332483,5.36644
9,-0.0774508,-0.139843,5.31879


Legendre polynomial: 8m49 s, `scale_factor = 1/3` and degree `30`, and 300 sample points.  

In [13]:
df_lege = solve_and_parse(pol_lege, x, f, TR, basis=:legendre)
sort!(df_lege, :z, rev=true)


=== Starting MSolve Parser (dimension: 2) ===
Processed 279 points (0.008s)


Row,x1,x2,z
,Float64,Float64,Float64
1,0.0259943,-0.140881,6.28934
2,0.287917,0.170829,6.16495
3,0.286793,0.251709,6.09483
4,0.0276141,0.331731,5.84256
5,0.0276342,0.327851,5.80976
6,0.027565,-0.288191,5.7418
7,0.286267,0.327073,5.42289
8,0.286258,0.33208,5.38275
9,-0.0777829,-0.139723,5.32538


In [1]:
using Optim
df_cheb.steps = zeros(nrow(df_cheb))
df_cheb.converged = zeros(nrow(df_cheb))

for i in 1:nrow(df_cheb)
    # println("Optimizing for point $i")
    x0 = [df_cheb.x1[i], df_cheb.x2[i]]
    res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=false))
    minimizer = Optim.minimizer(res)
    min_value = Optim.minimum(res)
    steps = res.iterations
    converged = Optim.converged(res)

    # df.dist_to_loc_min[i] = distance
    df_cheb.steps[i] = steps
    df_cheb.converged[i] = min_value
    summary(res)
end
df_cheb

UndefVarError: UndefVarError: `nrow` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [18]:
using GLMakie
function plot_polyapprox_rotate(pol::ApproxPoly, TR::test_input, df::DataFrame)
    # Extract coordinates and function values
    coords = pol.scale_factor * pol.grid .+ TR.center'
    z_coords = pol.z

    if size(coords)[2] == 2  # Plot if the dimensions are 2
        fig = Figure(size=(800, 600))
        ax = Axis3(fig[1, 1], title="Trefethen Function",
            xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

        # Create a regular grid for surface plotting
        x_unique = sort(unique(coords[:, 1]))
        y_unique = sort(unique(coords[:, 2]))

        # Initialize the Z grid with NaN
        Z = fill(NaN, (length(y_unique), length(x_unique)))

        # Fill the Z grid by matching coordinates
        for (idx, (x, y, z)) in enumerate(zip(coords[:, 1], coords[:, 2], z_coords))
            i = findlast(≈(y), y_unique)
            j = findlast(≈(x), x_unique)
            if !isnothing(i) && !isnothing(j)
                Z[j, i] = z
            end
        end

        # Create surface plot
        surface!(ax, x_unique, y_unique, Z,
            colormap=:viridis,
            transparency=true,
            alpha=0.8)

        # Plot the critical points
        scatter!(ax, df.x1, df.x2,
            df.z,
            markersize=10,
            color=:orange,
            label="Chebyshev approximant critical points")

        # Record rotating animation
        record(fig, "trefethern_rotation_d30.mp4", 1:240; framerate=30) do frame
            ax.azimuth[] = 1.7pi + 0.4 * sin(2pi * frame / 240)
            ax.elevation[] = pi / 4 + 0.3 * cos(2pi * frame / 240)
        end

        display(fig)
        return fig
    end
end

plot_polyapprox_rotate (generic function with 1 method)

In [19]:
plot_polyapprox_rotate(pol_cheb, TR, df_cheb)

In [20]:
GLMakie.closeall()